Assignment 6.2 

Anna Ramji

May 13, 2024

In [1]:
import duckdb
import pandas as pd

In [2]:
conn = duckdb.connect("database.db")
c = conn.cursor()

1. Create a query that will return the distinct species for which there is egg data (not all species and not all nests have egg data), so that you can then loop over those species. Your query should return two columns, species code and scientific name. Please order the results in alphabetic order of scientific name.

In [3]:
species_query = """
SELECT Code, Scientific_name
FROM Species s
INNER JOIN Bird_nests n ON s.Code = n.Species
INNER JOIN Bird_eggs e ON n.Nest_ID = e.Nest_ID
GROUP BY s.Code, s.Scientific_name
ORDER BY s.Scientific_name;
"""

In [8]:
# define loop with query, using .fetchall()
for row in c.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]

    egg_query = """
    SELECT Width*Width*Length AS Volume
    FROM Bird_eggs
    WHERE Nest_ID IN (
        SELECT Nest_ID
        FROM Bird_nests
        WHERE Species = ?
    );
    """
# directly load the results into a dataframe
    df = pd.read_sql(egg_query, conn, params=(species_code,))
    
    # calculate variation
    cv = round(df.Volume.std() / df.Volume.mean() * 100, 2)
    
    # output scientific name and cv
    print(f"{scientific_name} {cv}%")

Arenaria interpres 21.12%
Calidris alpina 5.46%
Calidris fuscicollis 16.77%
Charadrius semipalmatus 8.99%
Phalaropus fulicarius 4.65%
Pluvialis dominica 19.88%
Pluvialis squatarola 6.94%


/var/folders/f0/hghycvd17md1j6yxt_xy041h0000gn/T/ipykernel_9664/4092251057.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, conn, params=(species_code,))


In [9]:
conn.close()
